## 실제 활용해보며 익히는 MySQL 기본 + (중급) 문법
- 다양한 문법을 나열하기보다는, 실제 예제를 통해, 필요할 때, 그때 그때 문법을 익혀야 이해가 빠릅니다.

### 1. Schema 정의
- TABLE 분리, FOREIGN KEY, PRIMARY KEY 사용

In [ ]:
CREATE TABLE items (
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);

In [ ]:
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);

##### 참고: 한글 처리에 문제가 있을 경우, DB, TABLE 에 DEFAULT CHARSET=utf8 COLLATE=utf8_bin 옵션을 모두 추가해서 실행
```sql
CREATE DATABASE bestproducts DEFAULT CHARSET=utf8 COLLATE=utf8_bin; 
CREATE TABLE tablename(field definitions) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
```

In [7]:
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='funcoding', db='bestproducts', charset='utf8')
cursor = db.cursor()

sql = '''
CREATE TABLE items (
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);
'''
cursor.execute(sql)

sql = '''
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(10) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
'''
cursor.execute(sql)

db.commit()
db.close()

OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

### 2. 크롤링

#### main category 정보 가져오기

In [8]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    print ('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

http://corners.gmarket.co.kr//Bestsellers ALL
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G01 패션의류
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G02 신발/잡화
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G03 화장품/헤어
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G04 유아동/출산
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G07 식품
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G08 생활/주방/건강
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G09 가구/침구
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G05 스포츠/자동차
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G06 컴퓨터/전자
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G10 도서/음반
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G11 여행
http://corners.gmarket.co.kr//Bestsellers?viewType=G&groupCode=G12 e쿠폰/티켓


#### main/sub category 정보 가져오기

In [9]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

http://corners.gmarket.co.kr//Bestsellers ALL
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[에그머니](카드가능)(에그머니) 온라인 게임상품권 5만원', 'ori_price': '50000', 'dis_price': '45000', 'discount_percent': '10', 'item_code': '1791565626&ver', 'provider': ''}


NameError: name 'cursor' is not defined

In [ ]:
def get_category(category_link, category_name):
    print (category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    sub_categories = soup.select('div.navi.group ul li a')
    for sub_category in sub_categories:
        print (category_link, category_name, sub_category.get_text(), 'http://corners.gmarket.co.kr/' + sub_category['href'])
    

### main/sub category + 상품 정보 + 상품 코드 + 판매자(제공자) 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='funcoding', db='bestproducts', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

In [ ]:
def get_category(category_link, category_name):
    print (category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')

    get_items(soup, category_name, "ALL")
    
    sub_categories = soup.select('div.navi.group ul li > a')  # li 바로 밑 a 를 추출해라라는 뜻
    for sub_category in sub_categories:
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text())            

In [ ]:
def get_items(html, category_name, sub_category_name):
    items_result_list = list()
    best_item = html.select('div.best-list')
    for index, item in enumerate(best_item[1].select('li')):
        data_dict = dict()
        
        ranking = index + 1
        title = item.select_one('a.itemname')
        ori_price = item.select_one('div.o-price')
        dis_price = item.select_one('div.s-price strong span')
        discount_percent = item.select_one('div.s-price em')

        if ori_price == None or ori_price.get_text() == '':
            ori_price = dis_price
        
        if dis_price == None:
            ori_price, dis_price = 0, 0
        else:
            ori_price = ori_price.get_text().replace(',', '').replace('원', '')
            dis_price = dis_price.get_text().replace(',', '').replace('원', '')

        if discount_percent == None or discount_percent.get_text() == '':
            discount_percent = 0
        else:
            discount_percent = discount_percent.get_text().replace('%', '')
        
        product_link = item.select_one('div.thumb > a')
        item_code = product_link.attrs['href'].split('=')[1]
        
        res = requests.get(product_link.attrs['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > a > strong')
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
            
        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['discount_percent'] = discount_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider        
        
        save_data(data_dict)
        # print (category_name, sub_category_name, ranking, item_code, provider, title.get_text(), ori_price, dis_price, discount_percent)

### INSERT SQL 만들기

##### Table 구조
```sql
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(20) NOT NULL,
    sub_category VARCHAR(20) NOT NULL,
    item_ranking TINYINT NOT NULL,
    item_code VARCHAR(10) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
```

In [ ]:
item_info = {'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[오설록]프리미엄 티 컬렉션 40입 추석선물세트/쇼핑백지급', 'ori_price': '25000', 'dis_price': '18900', 'discount_percent': '24', 'item_code': '1164687842', 'provider': '오설록본사직영몰'}

In [ ]:
sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('
""" + item_info['category_name'] + """',
'""" + item_info['sub_category_name'] + """', 
'""" + str(item_info['ranking']) + """', 
'""" + item_info['item_code'] + """')"""       

In [10]:
sql.replace('\n', '')

NameError: name 'sql' is not defined

### 목적
- 실제로 데이터베이스를 사용하는 방법을 현업스타일로 설명을 드리는 것
- 현실 세계의 데이터를 어떻게 테이블로 정의할 것인가
- 어떻게 데이터를 넣을 것인가? (프로그래밍으로 넣는 것이 일반적)
- 저장된 데이터를 분석한다. (SQL만 쓰는 경우 + 프로그래밍에서 데이터를 분석해서, 풀스택으로 서비스화함)

##### 기존 SQL 강의: 단순히 세세한 SQL 문법을 다 소개받고, 끝

#### Table 구조
```sql
CREATE TABLE items (
    item_code VARCHAR(10) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);
```

In [11]:
sql = """INSERT INTO items VALUES('
""" + item_info['item_code'] + """',
'""" + item_info['title'] + """', 
""" + item_info['ori_price'] + """, 
""" + item_info['dis_price'] + """, 
""" + item_info['discount_percent'] + """, 
'""" + item_info['provider'] + """')"""    

NameError: name 'item_info' is not defined

In [12]:
sql.replace('\n', '')

NameError: name 'sql' is not defined

#### DELETE TABLE DATA

In [13]:
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='funcoding', db='bestproducts', charset='utf8')
cursor = db.cursor()

sql = "DELETE FROM ranking"
cursor.execute(sql)

sql = "DELETE FROM items"
cursor.execute(sql)

db.commit()
db.close()

OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

#### COUNT SQL 
- COUNT: 검색 결과의 row 수를 가져올 수 있는 SQL 문법
- SQL 예제: SELECT COUNT(*) FROM items

```sql
sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
cursor.execute(sql)
result = cursor.fetchone()
print (result[0])
```

- COUNT SQL 예제1: SELECT COUNT(*) FROM items (전체 row 수)
- COUNT SQL 예제2: SELECT COUNT(ori_price) FROM items (ori_price field 값이 있는 row 수)

### 최종 코드 (pymysql + crawling)

In [14]:
import requests
from bs4 import BeautifulSoup
import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='011936', db='bestproducts', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

db.commit()
db.close()

http://corners.gmarket.co.kr//Bestsellers ALL
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[에그머니](카드가능)(에그머니) 온라인 게임상품권 5만원', 'ori_price': '50000', 'dis_price': '45000', 'discount_percent': '10', 'item_code': '1791565626&ver', 'provider': ''}
0
INSERT INTO items VALUES('1791565626&ver',
        '[에그머니](카드가능)(에그머니) 온라인 게임상품권 5만원', 
        50000, 
        45000, 
        10, 
        '')
INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'ALL', 
    '1', 
    '1791565626&ver')
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 2, 'title': '[디스커버리](공용) 비글 V2 DXSHE1031DXSHE2031', 'ori_price': '54500', 'dis_price': '40330', 'discount_percent': '26', 'item_code': '2021114418&ver', 'provider': ''}
0
INSERT INTO items VALUES('2021114418&ver',
        '[디스커버리](공용) 비글 V2 DXSHE1031DXSHE2031', 
        54500, 
        40330, 
        26, 
        '')
INSERT INTO ranking (main_category, sub_category, item_rankin

{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 17, 'title': '[아웃백스테이크하우스](아웃백/빕스) Pays 외식/카페 결합 금액권 5만원권', 'ori_price': '50000', 'dis_price': '44500', 'discount_percent': '11', 'item_code': '2012122098&ver', 'provider': ''}
0
INSERT INTO items VALUES('2012122098&ver',
        '[아웃백스테이크하우스](아웃백/빕스) Pays 외식/카페 결합 금액권 5만원권', 
        50000, 
        44500, 
        11, 
        '')
INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'ALL', 
    '17', 
    '2012122098&ver')
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 18, 'title': '이니스프리 폼 발렌타인 기획전 UP TO 50', 'ori_price': '40000', 'dis_price': '28000', 'discount_percent': '30', 'item_code': '1636980511&ver', 'provider': ''}
0
INSERT INTO items VALUES('1636980511&ver',
        '이니스프리 폼 발렌타인 기획전 UP TO 50', 
        40000, 
        28000, 
        30, 
        '')
INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'ALL', 
   

KeyboardInterrupt: 

In [ ]:
def get_category(category_link, category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')

    get_items(soup, category_name, "ALL")

    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories:
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        print(category_name, sub_category.get_text())
        get_items(soup, category_name, sub_category.get_text())

In [ ]:
def get_items(html, category_name, sub_category_name):
    # 일부 서브 카테고리의 경우, 이벤트 페이지와 같이 표시될 수 있으므로, 해당 케이스들은 skip 함 (2020.09.30)
    best_item = html.select('div.best-list')
    
    if len(best_item[1].select('li')) > 0:
        for index, item in enumerate(best_item[1].select('li')):
            data_dict = dict()

            ranking = index + 1
            title = item.select_one('a.itemname')
            ori_price = item.select_one('div.o-price')
            dis_price = item.select_one('div.s-price strong span')
            discount_percent = item.select_one('div.s-price em')

            if ori_price == None or ori_price.get_text() == '':
                ori_price = dis_price

            if dis_price == None:
                ori_price, dis_price = 0, 0
            else:
                ori_price = ori_price.get_text().replace(',', '').replace('원', '')
                dis_price = dis_price.get_text().replace(',', '').replace('원', '')

            if discount_percent == None or discount_percent.get_text() == '':
                discount_percent = 0
            else:
                discount_percent = discount_percent.get_text().replace('%', '')

            product_link = item.select_one('div.thumb > a')
            item_code = product_link.attrs['href'].split('=')[1]

            res = requests.get(product_link.attrs['href'])
            soup = BeautifulSoup(res.content, 'html.parser')
            provider = soup.select_one('div.item-topinfo_headline > p > a > strong')
            if provider == None:
                provider = ''
            else:
                provider = provider.get_text()

            data_dict['category_name'] = category_name
            data_dict['sub_category_name'] = sub_category_name
            data_dict['ranking'] = ranking
            data_dict['title'] = title.get_text()
            data_dict['ori_price'] = ori_price
            data_dict['dis_price'] = dis_price
            data_dict['discount_percent'] = discount_percent
            data_dict['item_code'] = item_code
            data_dict['provider'] = provider

            save_data(data_dict)

In [ ]:
def save_data(item_info):
    print (item_info)
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
    cursor.execute(sql)
    result = cursor.fetchone()
    print (result[0])
    if result[0] == 0: # 상품이 없을 때
        sql = """INSERT INTO items VALUES('""" + item_info['item_code'] + """',
        '""" + item_info['title'] + """', 
        """ + str(item_info['ori_price']) + """, 
        """ + str(item_info['dis_price']) + """, 
        """ + str(item_info['discount_percent']) + """, 
        '""" + item_info['provider'] + """')"""
        cursor.execute(sql)
    
    sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('""" + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """', 
    '""" + str(item_info['ranking']) + """', 
    '""" + item_info['item_code'] + """')"""      
    cursor.execute(sql)